# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [1]:
import pandas as pd
import json
import os
import random
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional
from langchain_tavily import TavilySearch
from langchain.schema import Document
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
load_dotenv()

True

In [2]:
azure_llm = AzureChatOpenAI(
    azure_endpoint="https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
    api_key=os.environ["AZURE_API_KEY"],
    model_name="gpt-4o",
    api_version="2025-01-01-preview",
    temperature=0.5,
)

aws_llm = ChatBedrock(
    # model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.5),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

In [3]:
# Import N3 Vocabulary
file_path = '../../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()
vocab_dict = words.set_index(words.columns[0])[words.columns[1]].to_dict()
vocab_dict = json.dumps(vocab_dict, ensure_ascii=False, separators=(',', ':'))

#### load Models

#### Exam Paper Outline
### A. overall thinking the structure of an exam
1. distribution of the difficulty 
2. topics
3. reasoning

## Data Strcuture

# Kanji 读假名（读音问题）

In [4]:
def online_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """
    
    print("---WEB SEARCH---")
    
    topic = state['messages'][0].content
    
    tavily_search_tool = TavilySearch(
        max_results=5,
        topic="news",
        days=1
    )
    # Web search
    docs = tavily_search_tool.invoke({"query": topic})
    
    print(docs)

    web_results = "\n".join([d["content"] for d in docs["results"]])
    
    print("Web results: ", web_results)

    return {"documents": web_results, "topic": topic}

In [5]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    documents: str
    messages: Annotated[list, add_messages]


example = """
15. 大雪で朝から電車が（　）している。
	1.	縮小
	2.	滞在
	3.	延期
	4.	運休

16. 今日は暑かったので、シャツが（　）でぬれてしまった。
	1.	いびき
	2.	あくび
	3.	あせ
	4.	いき

17. 答えさんに声がよく聞こえるように、（　）を使って話してください。
	1.	サイレン
	2.	エンジン
	3.	ノック
	4.	マイク

18. 昨日は早く寝たが、夜中に大きな音がして目が（　）しまった。
	1.	嫌がって
	2.	覚めて
	3.	驚いて
	4.	怖がって

19. 林さんはいつも冗談ばかり言うので、その話も本当かどうか（　）。
	1.	あやしい
	2.	おそろしい
	3.	にくらしい
	4.	まずしい

20. 本日の面接の結果は、1 週間以内にメールで（　）します。
	1.	広告
	2.	合図
	3.	通知
	4.	伝言

21. 兄はいつも（　）シャツを着ているので、遠くにいてもすぐに見つかる。
	1.	派手な
	2.	盛んな
	3.	わがままな
	4.	身近な

22. ここに車を止めることは規則で（　）されていますから、すぐに移動してください。
	1.	支配
	2.	英殺
	3.	禁止
	4.	批判

23. このコートは古いがまだ着られるので、捨ててしまうのは（　）。
	1.	もったいない
	2.	しかたない
	3.	かわいらしい
	4.	こいかない

24. 弟への誕生日プレゼントは、誕生日まで弟に見つからないように、たんすの奥に（　）。
	1.	包んだ
	2.	隠した
	3.	囲んだ
	4.	閉じた

25. 山口さんは今度のパーティーには来られないかもしれないが、（　）誘うつもりだ。
	1.	十分
	2.	一応
	3.	けっこう
	4.	たいてい
"""

# Nodes
def question_draft_generator(state: QuestionState):
    """First LLM call to generate initial question"""
    print("---Generator----")
        
    search_result = state['documents'],
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """
                    You are a Japanese teacher. Your job is to write a vocabulary question for candidates to identify the correct kanji writing of a given word in hiragana for a JLPT N3 level exam paper. The question format follows:
                Each question presents a word in hiragana within a sentence, and candidates must choose the correct kanji representation from four options. The options should include one correct kanji form and three distractors that are plausible but incorrect. The JLPT exam paper includes a mix of easy, moderate, and difficult questions to accurately assess the test-taker’s proficiency across different aspects of the language.
                    The vocabulary should be restricted to N3 level, use the vocabulary in the `Dictionary` as much as you can.
                    Please refer to the question examples following the formal exam paper. please highlight the word to ask candidate with <u><em></em></u>.
                    Append the correct answer and explanation of the main challenges on why the teacher asks this question to the candidate in simplified Chinese at each question.
                    Finally, output beautiful markdown format.
                    Dictionary: {vocab_dict}
                    Search result: {search_result}
                    Formal exam paper: {example}
                """
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    
    input = { "topic" : state['topic'],
              "search_result": search_result, 
              "vocab_dict": vocab_dict,
              "example": example,
              "messages": state["messages"]
              }
    # final_message = prompt.format_messages(**input)
    # print(final_message)
    
    generate = prompt | azure_llm
    
    msg = generate.invoke(input=input)
    
    
    return {"question": msg.content, "messages": [AIMessage(content=msg.content)] }


def reflection_node(state: QuestionState) -> QuestionState:
    print("---REVISOR---")
    
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [state["messages"][0]] + [
        cls_map[msg.type](content=msg.content) for msg in state["messages"][1:]
    ]

    reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """you are a Japanese language educator reviewing a JLPT exam paper. Generate critique and recommendations for the user's submission.
            the review focuses on content accuracy and question quality. 
            - For content accuracy, you must verify that the grammar and vocabulary questions accurately reflect the appropriate JLPT N3 level, ensuring the reading passages are clear, relevant, and appropriately challenging. 
            - For question quality, you must ensure all questions are clearly worded and free from ambiguity to comprehensively assess different language skills, and confirm that the difficulty level of the questions matches the intended JLPT N3 level.
            - During detailed refinement, you check the format and presentation of the paper, ensuring it is well-organized and the instructions are clear and concise. you also ensure the content is culturally appropriate and relevant to Japanese language and culture.
            - Finally, you make give feedback, providing detailed recommendations, including requests.If you think the exam paper is good enough, you just say "GOOD ENOUGH"
            """
        ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    reflect = reflection_prompt | azure_llm
    
    res = reflect.invoke(translated)
    
    print(res.content)
    
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=res.content)]}



In [6]:
# Build workflow
builder = StateGraph(QuestionState)

builder.add_node("online_search", online_search)
builder.add_node("generator", question_draft_generator)
builder.add_node("reflector", reflection_node)
# Add nodes

def should_continue(state: QuestionState):
    if state["messages"]:
        if len(state["messages"]) > 6: 
            print("--- Reach the Maximum Round ---")
            return END
        elif "GOOD ENOUGH" in state["messages"][-1].content:
            print("--- AI Reviser feels Good Enough ---")
            return END
    return "generator"

# Add edges to connect nodes
builder.add_edge(START, "online_search")
builder.add_edge("online_search", "generator")
builder.add_edge("generator","reflector")
# 
builder.add_conditional_edges("reflector", should_continue)
memory = MemorySaver()

# Compile
kanji_graph = builder.compile()

# Show workflow
# display(Image(kanji_graph.get_graph().draw_png()))

In [7]:
row = words.iloc[1]
word = f"{row[0]}({row[1]})"
word

C:\Users\HW-BK003\AppData\Local\Temp\ipykernel_16172\846002302.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  word = f"{row[0]}({row[1]})"


'用心(ようじん)'

In [8]:
# # Debug the Conversation
# for event in kanji_graph.stream(
#     {
#         "messages": [
#             HumanMessage(
#                 content=word
#             )
#         ],
#     },
#     config={"configurable": {"thread_id": "1"}},
# ):
#     print(event)
#     print("---")

In [9]:
kanji = kanji_graph.invoke(
    {
       "messages": [
                HumanMessage(
                    content=word
                )
            ],
        },
    config={"configurable": {"thread_id": "1"}}
)
display(Markdown(kanji["question"]))

---WEB SEARCH---
{'query': '用心(ようじん)', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '用心／要心（ようじん）とは？ 意味・読み方・使い方をわかりやすく解説 - goo国語辞書', 'url': 'https://dictionary.goo.ne.jp/word/用心/', 'content': '用心／要心（ようじん）とは。意味や使い方、類語をわかりやすく解説。[名](スル)1 心をくばること。気をつけること。「風邪をひかないように—する」2 万一に備えて注意・警戒を怠らないこと。「火の—」「—の悪い家」 - goo国語辞書は30万9千件語以上を収録。', 'score': 0.82117355, 'raw_content': None}, {'title': '用心 - Wiktionary, the free dictionary', 'url': 'https://en.wiktionary.org/wiki/用心', 'content': '用心しない: ようじんしない: yōjin shinai: Negative continuative 用心せず: ようじんせず: yōjin sezu: Formal 用心します: ようじんします: yōjin shimasu: Perfective 用心した: ようじんした: yōjin shita: Conjunctive 用心して: ようじんして: yōjin shite: Hypothetical conditional 用心すれば', 'score': 0.79769903, 'raw_content': None}, {'title': '「用心」とは？意味や例文や読み方や由来について解説!｜コトバスタ', 'url': 'https://kotobasta.com/13435/', 'content': '「用心」は「ようじん」と読みます。 この言葉は、特に日本語において非常に重要な役割を持っています。 特に何かに注意を払う必要があるときや、危険を察知したときに自然と使われる表現です。', 'score': 0.7568076, 'raw_conte

### Vocabulary Question: 用心 (ようじん)

#### Question:
風邪をひかないように、しっかり<u><em>ようじん</em></u>してください。

1. 用心  
2. 注意  
3. 要心  
4. 留意  

---

#### Correct Answer:
**1. 用心**

---

#### Explanation (简体中文):
正确答案是“用心”。“用心”表示小心、注意，特别是在防止某些危险或问题时使用。其他选项如“注意”和“留意”虽然也有类似的含义，但它们通常用于更具体或不同的上下文；“要心”是错误的写法，不是标准日语词汇。

这道题的难点在于考察考生对日语中近义词的细微区别的掌握，以及对正确汉字的选择能力。这是JLPT N3考试中常见的题型，旨在测试考生的词汇和汉字知识。

---